In [10]:
from extractors.pdf_extractor import extract_pdf_as_markdown
from chains.audit_chain import analyze_module
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
import os
from dotenv import load_dotenv
import asyncio
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [2]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/Business Continuity Plan E-WISE [Restricted].docx.pdf"
controls = pd.read_json("ISO_27001_2022_Controls_List.json")


In [3]:
controls

,Section,Control ID,Control Title
0,Clause 4,4.1,Understanding the organization and its context
1,Clause 4,4.2,Understanding the needs and expectations of in...
2,Clause 4,4.3,Determining the scope of the ISMS
3,Clause 4,4.4,Information security management system
4,Clause 5,5.1,Leadership and commitment
...,...,...,...
110,A.8 (Technological Controls),A.8.30,Control title placeholder for A.8.30
111,A.8 (Technological Controls),A.8.31,Control title placeholder for A.8.31
112,A.8 (Technological Controls),A.8.32,Control title placeholder for A.8.32
113,A.8 (Technological Controls),A.8.33,Control title placeholder for A.8.33


In [4]:
n = len(controls)
part_size = n // 3

df1 = controls.iloc[:part_size]
df2 = controls.iloc[part_size:2*part_size]
df3 = controls.iloc[2*part_size:]

# Optional: reset index
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)

In [5]:
text = extract_pdf_as_markdown(file_path)


✔️ Tesseract detected: tesseract 4.1.1


In [ ]:

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3]
audits = []
results = []
for clause in clauses:
    audit_chain = analyze_module(llm)
    results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



✔️ Tesseract detected: tesseract 4.1.1


/tmp/ipykernel_13772/3417714339.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
/home/sherry/DocAnalyzer/Document_Analyzer_Streamlit_App/chains/audit_chain.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_13772/3417714339.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results.append(audit_chain.run(text=te

In [13]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1, df2, df3]
results = []
total_tokens = 0
total_cost = 0.0

async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())


✔️ Tesseract detected: tesseract 4.1.1
🔎 Clause Processed: 38 controls
🧠 Tokens used: 10602
💵 Cost: $0.008219
🔎 Clause Processed: 38 controls
🧠 Tokens used: 11004
💵 Cost: $0.008605
🔎 Clause Processed: 39 controls
🧠 Tokens used: 11123
💵 Cost: $0.008756

✅ All clauses processed (async).
🔢 Total tokens used: 32729
💰 Total cost: $0.025580


In [16]:
results = final_results

In [17]:
import json
import pandas as pd

# Parse each JSON string and collect all items into a single list
all_items = []
for js in results:
    try:
        # Remove optional code formatting like ```json and ```
        clean_js = js.strip().removeprefix("```json").removesuffix("```").strip()
        all_items.extend(json.loads(clean_js))
    except json.JSONDecodeError as e:
        print("Failed to parse:", js)
        raise e

# Convert to DataFrame
resutls_df = pd.DataFrame(all_items)



In [18]:
resutls_df

,Section,Control Id,Control Title,Compliance,Gaps Identified,Recommended Action
0,4,4.1,Understanding the organization and its context,🟡,The document indirectly references the organiz...,Include a formal section describing the organi...
1,4,4.2,Understanding the needs and expectations of in...,🟡,The document mentions communication responsibi...,Add a detailed identification and analysis of ...
2,4,4.3,Determining the scope of the ISMS,🟡,The document states the Business Continuity Pl...,"Formally define and document the ISMS scope, i..."
3,4,4.4,Information security management system,❌,The document focuses on business continuity an...,"Include a description of the ISMS framework, p..."
4,5,5.1,Leadership and commitment,🟡,Leadership roles and responsibilities are desc...,Document leadership commitment statements and ...
...,...,...,...,...,...,...
110,A.8 (Technological Controls),A.8.30,Control title placeholder for A.8.30,❌,Control A.8.30 is not addressed.,Add relevant technological controls for A.8.30.
111,A.8 (Technological Controls),A.8.31,Control title placeholder for A.8.31,❌,No mention of A.8.31 controls.,Document and implement controls for A.8.31.
112,A.8 (Technological Controls),A.8.32,Control title placeholder for A.8.32,❌,No evidence of controls for A.8.32.,Add controls and procedures for A.8.32.
113,A.8 (Technological Controls),A.8.33,Control title placeholder for A.8.33,❌,No mention of A.8.33 controls.,Include controls for A.8.33.


In [20]:
resutls_df.to_excel("result.xlsx")

In [21]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'
